In [100]:
import torch
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import Normalize
from torchmetrics import Accuracy

import torch.optim as optim
from cleverhans.torch.attacks.projected_gradient_descent import (projected_gradient_descent)

import quantus
import captum
from captum.attr import Saliency, IntegratedGradients, NoiseTunnel

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random
import copy
import gc

import warnings
warnings.filterwarnings('ignore')

In [102]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [104]:
transformer = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), Normalize(mean=0.3814, std=0.3994)])

train_dataset = datasets.FashionMNIST(root='./datasets', train=True, download = True, transform=transformer)
test_dataset = datasets.FashionMNIST(root='./datasets', train=False, download = True, transform=transformer)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True) # num_workers=4,
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, pin_memory=True)

In [106]:
#for natural and adversarial LeNet Model 
class LeNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_1 = torch.nn.Conv2d(1, 6, 5)
        self.pool_1 = torch.nn.MaxPool2d(2, 2)
        self.relu_1 = torch.nn.ReLU()
        self.conv_2 = torch.nn.Conv2d(6, 16, 5)
        self.pool_2 = torch.nn.MaxPool2d(2, 2)
        self.relu_2 = torch.nn.ReLU()
        self.fc_1 = torch.nn.Linear(256, 120)
        self.relu_3 = torch.nn.ReLU()
        self.fc_2 = torch.nn.Linear(120, 84)
        self.relu_4 = torch.nn.ReLU()
        self.fc_3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool_1(self.relu_1(self.conv_1(x)))
        x = self.pool_2(self.relu_2(self.conv_2(x)))
        x = x.view(x.shape[0], -1)
        x = self.relu_3(self.fc_1(x))
        x = self.relu_4(self.fc_2(x))
        x = self.fc_3(x)
        return x

In [108]:
model = LeNet()
learning_rate = 0.001
epochs = 50 
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [110]:
def evaluate_model(model, data, device):
    model.eval()
    logits = torch.Tensor().to(device)
    targets = torch.LongTensor().to(device)

    with torch.no_grad():
        for x_batch, y_batch in data:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            logits = torch.cat([logits, model(x_batch)])
            targets = torch.cat([targets, y_batch])
    
    return torch.nn.functional.softmax(logits, dim=1), targets

In [112]:
def train_normal(model, epochs):
    model.train()
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            logits = model(x_batch)
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [77]:
model_normal = train_normal(model = model.to(device), epochs = epochs)

Epoch 1/50 - test accuracy: 84.84% and CE loss 0.39
Epoch 2/50 - test accuracy: 86.71% and CE loss 0.66
Epoch 3/50 - test accuracy: 88.12% and CE loss 0.52
Epoch 4/50 - test accuracy: 87.97% and CE loss 0.16
Epoch 5/50 - test accuracy: 88.17% and CE loss 0.37
Epoch 6/50 - test accuracy: 88.92% and CE loss 0.05
Epoch 7/50 - test accuracy: 88.87% and CE loss 0.32
Epoch 8/50 - test accuracy: 87.97% and CE loss 0.34
Epoch 9/50 - test accuracy: 89.63% and CE loss 0.42
Epoch 10/50 - test accuracy: 89.14% and CE loss 0.12
Epoch 11/50 - test accuracy: 88.67% and CE loss 0.05
Epoch 12/50 - test accuracy: 89.57% and CE loss 0.37
Epoch 13/50 - test accuracy: 89.65% and CE loss 0.07
Epoch 14/50 - test accuracy: 90.19% and CE loss 0.33
Epoch 15/50 - test accuracy: 89.70% and CE loss 0.23
Epoch 16/50 - test accuracy: 89.77% and CE loss 0.13
Epoch 17/50 - test accuracy: 89.27% and CE loss 0.13
Epoch 18/50 - test accuracy: 89.28% and CE loss 0.12
Epoch 19/50 - test accuracy: 89.38% and CE loss 0.20
Ep

In [79]:
# Model to GPU and eval mode.
model_normal.to(device)
model_normal.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_normal, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 88.92%


In [81]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "lenet_fmnist.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving the model: {MODEL_SAVE_PATH}")
torch.save(obj=model_normal.state_dict(), f=MODEL_SAVE_PATH)

Saving the model: models\lenet_fmnist.pth


# Train Adversarial Models

In [114]:
model = LeNet()
learning_rate = 1e-4
epochs = 40
eps = [0.01,0.03,0.06,0.0,0.3,0.5]
criterion = torch.nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [116]:
def evaluate_model(model, data, device):
    model.eval()
    logits = torch.Tensor().to(device)
    targets = torch.LongTensor().to(device)

    with torch.no_grad():
        for x_batch, y_batch in data:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            logits = torch.cat([logits, model(x_batch)])
            targets = torch.cat([targets, y_batch])
    
    return torch.nn.functional.softmax(logits, dim=1), targets

In [118]:
def train_adv(model, epsilon, epochs):
    model.train()
    eps = epsilon
    for epoch in range(epochs):
        for x_batch, y_batch in train_dataloader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            x_batch = projected_gradient_descent(model, x_batch, eps, eps/10, 40, np.inf)
            optimizer.zero_grad()
            logits = model(x_batch)
            loss = criterion(logits, y_batch)
            loss.backward()
            optimizer.step()

        # Evaluate model!
        if epochs%10==0:
            predictions, labels = evaluate_model(model, test_dataloader, device)
            test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())
            print(f"Epoch {epoch+1}/{epochs} - test accuracy: {(100 * test_acc):.2f}% and CE loss {loss.item():.2f}")
    return model

In [120]:
model_adversarial = train_adv(model=model.to(device),
                    epsilon = eps[0], 
                    epochs=epochs)

Epoch 1/40 - test accuracy: 74.91% and CE loss 0.61
Epoch 2/40 - test accuracy: 79.06% and CE loss 0.47
Epoch 3/40 - test accuracy: 81.38% and CE loss 0.84
Epoch 4/40 - test accuracy: 82.89% and CE loss 0.35
Epoch 5/40 - test accuracy: 83.80% and CE loss 0.48
Epoch 6/40 - test accuracy: 84.79% and CE loss 0.23
Epoch 7/40 - test accuracy: 84.83% and CE loss 0.18
Epoch 8/40 - test accuracy: 85.29% and CE loss 0.33
Epoch 9/40 - test accuracy: 86.13% and CE loss 0.33
Epoch 10/40 - test accuracy: 86.52% and CE loss 0.37
Epoch 11/40 - test accuracy: 86.38% and CE loss 0.56
Epoch 12/40 - test accuracy: 87.10% and CE loss 0.11
Epoch 13/40 - test accuracy: 86.97% and CE loss 0.42
Epoch 14/40 - test accuracy: 87.02% and CE loss 0.32
Epoch 15/40 - test accuracy: 87.43% and CE loss 0.17
Epoch 16/40 - test accuracy: 87.34% and CE loss 0.39
Epoch 17/40 - test accuracy: 88.18% and CE loss 0.14
Epoch 18/40 - test accuracy: 88.15% and CE loss 0.36
Epoch 19/40 - test accuracy: 88.11% and CE loss 0.22
Ep

In [126]:
# Model to GPU and eval mode.
model_adversarial.to(device)
model_adversarial.eval()

# Check test set performance.
predictions, labels = evaluate_model(model_adversarial, test_dataloader, device)
test_acc = np.mean(np.argmax(predictions.cpu().numpy(), axis=1) == labels.cpu().numpy())        
print(f"Model test accuracy: {(100 * test_acc):.2f}%")

Model test accuracy: 89.50%


In [128]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "lenet_fmnist_adv.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving the model: {MODEL_SAVE_PATH}")
torch.save(obj=model_adversarial.state_dict(), f=MODEL_SAVE_PATH)

Saving the model: models\lenet_fmnist_adv.pth
